In [2]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Прямая загрузка модели (если она уже обучена)
def load_model_for_inference():
    model_name = 'cointegrated/rubert-tiny2'
    
    # Пробуем загрузить как есть
    try:
        model = BertForSequenceClassification.from_pretrained(model_name, num_labels=4)
        tokenizer = BertTokenizer.from_pretrained(model_name)
        print("Модель загружена успешно")
        return model, tokenizer
    except:
        print("Не удалось загрузить модель. Используем базовую модель BERT...")
        # Альтернатива: используем базовую модель
        from transformers import AutoModelForSequenceClassification, AutoTokenizer
        model = AutoModelForSequenceClassification.from_pretrained(model_name)
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        return model, tokenizer

def analyze_emotions(texts):
    model, tokenizer = load_model_for_inference()
    model.eval()
    
    emotions = ['нейтральный', 'негативный', 'позитивный', 'смешанный']
    
    results = []
    for text in texts:
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
        
        with torch.no_grad():
            outputs = model(**inputs)
        
        probabilities = torch.softmax(outputs.logits, dim=1)
        predicted_class = torch.argmax(probabilities, dim=1).item()
        
        results.append({
            'text': text[:100] + '...' if len(text) > 100 else text,
            'predicted_emotion': emotions[predicted_class],
            'probabilities': probabilities.numpy()[0]
        })
    
    return results

# Ваши тексты
texts = [
    "Бои у Сопоцкина и Друскеник закончились отступлением германцев. Неприятель, приблизившись с севера к Осовцу начал артиллерийскую борьбу с крепостью. В артиллерийском бою принимают участие тяжелые калибры. С раннего утра 14 сентября огонь достиг значительного напряжения. Попытка германской пехоты пробиться ближе к крепости отражена. В Галиции мы заняли Дембицу. Большая колонна, отступавшая по шоссе от Перемышля к Саноку, обстреливалась с высот нашей батареей и бежала, бросив парки, обоз и автомобили. Вылазки гарнизона Перемышля остаются безуспешными. При продолжающемся отступлении австрийцев обнаруживается полное перемешивание их частей, захватываются новые партии пленных, орудия и прочая материальная часть. На перевале Ужок мы разбили неприятельский отряд, взяли его артиллерию и много пленных и, продолжая преследовать, вступили в пределы Венгрии.",           
    "Министерство народного просвещения, в виду происходящих чрезвычайных событий, признало соответственным в день годовщины со дня рождения М.Ю. Лермонтова (2-го октября 1914 года) ограничиться совершением в учебных заведениях панихиды по поэту, отложив празднование юбилея до более благоприятного времени.",           
    "Штабс-капитан П. Н. Нестеров на днях, увидев в районе Желтиева, в Галиции, летящий над нашим расположением австрийский аэроплан, собиравшийся бросить бомбы, взлетел на воздух, атаковал неприятеля и протаранил неприятельский аппарат, предотвратив жертвы в наших войсках. Сам Нестеров при этом погиб смертью героя. По словам доставленных в Киев пленных австрийских офицеров, всей неприятельской армии хорошо известно имя Нестерова. Во время воздушных разведок русских авиаторов австрийцы всегда безошибочно определяли, каким аппаратом управлял Нестеров. Когда показывался аэроплан-птица, красиво и вольно паривший в воздухе, австрийцы указывали — Das ist Nesteroff! Австрийцы боялись покойного, и все их усилия были направлены к прекращению его деятельности. За задержание отважного летчика была объявлена большая премия. Нестеров погиб в 27 лет. После Нестерова остались жена и двое детей — девочка, 5-ти лет, и мальчик, 3-х лет. Иллюстрированный журнал «Искры» № 35, сентябрь 1914 года  Песнь о Нестерове В безбрежности неба, в бескрайности ясной пустыниСражались лишь птицы и гибли лишь птицы доныне.Так птице подобный, полет свой направив могучий,Унесся пилот легкокрылый за темные тучи.А там, на земле, колыхаясь, знамена шумели, И лязгали сабли, и пушки зловеще гремели.И пламенно богу войны посылал он молитвы,И очи горели и звали и жаждали битвы…Величие духа в равнине обманчиво-зыбкой.Презрение смерти под светлою солнца улыбкой…Мольбы не напрасны. Велению рока послушный,Противник уж реет кругами в стихии воздушнойИ дрогнуло сердце. Рука, как пружина стальная,На руль налегла. Встрепенулась машина живая…Чти, родина, память героя! Решенье созрелоМгновенье – и врезался дерзко он в хрупкое тело.Пощады не знает стихия. Победа наградой…И пали противники грузной и страшной громадой…В безбрежности неба, в бескрайности ясной пустыниСражались лишь птицы и гибли лишь птицы доныне. Журнал «Нива» №37, сентябрь 1914 года",                  
    "Фотограф-корреспондент Daily Mirror рассказывает случай, который порадует всех друзей животных. Лейтенант бельгийской артиллерии, руководивший обороной одного из фортов Льежа, ни за что не хотел расстаться с своей собакой — бульдогом. Когда пруссаки пробрались между фортов в самый город, офицеру пришло в голову доверить бульдогу письмо, в котором он посылал успокоительную весть своим родителям. Благородный пес честно исполнил свою миссию. Десять часов спустя бульдог проник обратно в форт и принес ответ. С этого момента бульдог стал настоящим гонцом. Много раз пробирался он через линии германских войск, неся на себе спрятанными в ошейнике шифрованные депеши.",           
    "Лица, приехавшие в Варшаву из Люблина, передают, что туда доставлен «швабский зверь» — взятый в плен прусский майор Прейскер, бывший комендант Калиша. Это — здоровый детина, с типично прусским наглым лицом и мутными глазами. Когда этого «зверя» вели с партией пленных по улице и из толпы раздались крики и проклятия, Прейскер трусливо замешался в толпу пленных, стараясь скрыться. Зато в заключении, под конвоем наших солдат, Прейскер принял пришедшего к нему русского офицера сидя, нагло развалившись в кресле. Конвойные «помогли» ему подняться и вытянуться во фронт. Кроме Прейскера, нашими войсками взят в плен и другой «швабский зверь» — капитан Шмидт, живьем сжигавший жителей деревень, запирая их в хатах. Журнал «Нива» №38 от 20 сентября 1914 года",                   
    "Как стало известно агентству Ассошиэйтед Пресс, экипаж последней экспедиции на станцию ""Мир"" считает ее способной выйти из-под контроля.Командир Виктор Афанасьев сказал: ""Мы чувствуем себя хорошо, но грустим, что оставили станцию летать в беспилотном режиме."" Gazeta Ru подробно писала о том, что финансирование программы ""Мир"" закрыто, но конструкторы не спешат топить ее в Тихом океане, надеясь на изыскание внебюджетных средств.Они надеются, что правительство оплатит финальную экспедицию на ""Мир"" в феврале или марте 2000 года. Новая экспедиция должна постепенно перевести станцию на низкую орбиту, после чего 140- тонный ""Мир"" в основном сгорит в атмосфере, а незначительные его части упадут в Тихий океан. Но нет никакой гарантии, что пережившая столько отказов оборудования конструкция поведет себя адекватно командам. Афанасьев перечислил потенциально опасные элементы ""Мира"" на пресс- конференции, состоявшейся в понедельник в Звездном Городке. ""Может упасть давление в кабине,- сказал он,- и это станет серьезным препятствием для работы следующей экспедиции. Слишком велика роль ""человеческого фактора"" , элементарная ошибка может привести как к потере топлива, так и к потере ориентации."" Американское космическое агентство по аэронавтике и космическому пространству предлагало России затопить ""Мир"", а его ресурсы перевести на международную космическую станцию. Но Россия не хочет расставаться с легендарным символом.",     
    "В зале игровых автоматов в третьем ярусе подземного комплекса ""Охотный ряд"" на Манежной площади в центре Москвы около 20.00 прогремел мощный взрыв. По данным ФСБ, погибли не менее четырех человек, 20 человек ранены. Однако уточненная оценка числа пострадавших в результате этого взрыва может достигнуть ста человек. Агентство ИТАР-ТАСС, в сообщении от 21.15 со ссылкой на источники в ГУВД Москвы, говорит только о 30 раненых, в том числе о двух пострадавших в тяжелом состоянии. Однако число пострадавших в результате этого взрыва может составить до ста человек. По данным РИА ""Новости"", Боткинская больница, Институт им. Склифосовского, 1-ая Градская, 36-ая и 64-ая горбольница работают только на прием пострадавших. С места происшествия запросили 20 машин скорой помощи. Из торгового центра эвакуированы все посетитетели. Среди причин происшедшего называют террористический акт, связанный с последними событиями в Дагестане, однако, по сообщению ОРТ, не исключается и версия об аварии (взорвался игровой автомат). Более точных данных о причнах взрыва и количестве жертв представители УФСБ называть не торопятся.",                  
    "Япония приняла решение разморозить кредиты России на сумму в полтора миллиарда долларов. Об этом заявил сегодня после встречи с премьер-министром Японии вице-премьер правительства России Виктор Христенко. Он прибыл в Токио во главе делегации для участия в заседании  российско-японской межправительственной комиссии по торгово-экономическим  вопросам, которое  начнется в  среду  в Токио. Один из участников подготовки этого заседания сообщил ""Интерфаксу"", что Христенко постарается на конкретных макроэкономических показателях  за последние  месяцы доказать, что экономика России становится более стабильной и убедить Японию в своей платежеспособности и партнерской надежности. ""Если на заседании  комиссии японская сторона будет удовлетворена этими данными, то многие российско-японские проекты по улучшению  экономики дальневосточного побережья, примут более реальные очертания"", - подчеркнул источник. Сегодняшнее сообщение о разморозке кредита показывает, что приведенные Христенко выкладки оказались убедительными для японцев. В  отношении   Японии  еще   со  времен СССР  осталаськоммерческая задолженность  японским фирмам,  общий объем которой составляет около $1,1 миллиардов. Основная задолженность перед крупными фирмами, объединившимися  в Токийский клуб, составляет около $925 миллионов. Общий же объем внешнего долга России в настоящее время  составляет около $150 миллиардов, и платежи по его обслуживанию в соответствии  с графиком  должны были  бы составить  около  90% доходов федерального бюджета.",               
    "Британцы отмечают сегодня скорбную дату - вторуюгодовщину со дня смерти принцессы Дианы. Она погибла вавтомобильной катастрофе в парижском туннеле в ранние утренниечасы 31 августа 1997 года. С утра к золоченым воротамКенсингтонского дворца, где жила Диана, приходят ее поклонники.В чугунные завитки ограды вплетаются сотни букетов. Людиоставляют дары принцессе - коробки конфет, плюшевых мишек,открытки со посвященными ей стихами, стенгазеты. Вчера и сегодня болееТысячи англичан подписали петицию с требованием к властямускорить сооружение мемориала принцессе. Пока Мемориальныйкомитет собирается переоборудовать в ее честь детскую игровуюплощадку, которая находится на территории Кенсингтонского паркаи создать посвященную ей ""прогулочную дорожку"" вдоль похоронного маршрута. Мемориальный фонд имени принцессы Дианы собрал в видепожертвований населения более 85 миллонов фунтов (140 миллионов долларов).Средичленов правления фонда нет согласия относительно использованиясредств. Они заняты не столько помощью больным иобделенным, как завещала принцесса, сколько в основном судебнойсхваткой за авторские права на ее, Дианы, изображения. Королевская семья отметит траурную годовщину сдержаннойпоминальной церемонией в шотландском замке Балморал, где будутприсутствовать сыновья Дианы - 17-летний Вильям и 14-летнийГарри, их отец принц Чарльз. Родственники Дианы соберутсяотдельно в Олторпе, поместье ее брата графа Чарльза Спенсера. Впоместье создан мемориальный музей Дианы, который в этом годупосетило 142 тысячи человек. Ожидается, что в ближайшие дни будут оглашеныокончательные результаты расследования аварии в Париже.Британские газеты уже получили этот документ. Никаких сенсацийтам нет. Единственная причина катастрофы - опьянение водителя ипревышение скорости, пришли к заключению французскиеследователи. Содержание алкоголя в крови водителя лимузина Генри Пола в три раза превосходило норму- 1,8 грамма на литр. Фиат ""Уно"", якобы послуживший причиной трагедии, так и не найден- в Париже таких автомобилей более сорока тысяч. Не будет предъявлено никаких обвиненийфотографам-папарацци, преследовавшим ""Мерседес"" Дианы на мотоциклах. Письма Дианы к бывшему любовнику,майору Джеймсу Хьюитту, могут быть вскоре опубликованы отдельной книгой. Хьюитт нарушил клятву никогда не использовать для печати 65 писем принцессы, написанных ею после того, как ее брак с принцем Чарльзом развалился. В письмах королевская семья характеризуется весьма нелицеприятно.",              
    "С 1 сентября на всей территории России вводится новая форма таможенной декларации для физических лиц. Как пояснили в Государственном таможенном комитете (ГТК), форма декларации, которая была одобрена Советом руководителей таможенных служб государств-участников Содружества, является единой на территории всех стран СНГ, за исключением Армении. По данным ИТАР-ТАСС, в новой декларации содержится более подробный перечень товаров, подлежащих обязательному декларированию. Так, впервые в их число включены ядовитые и сильно действующие вещества и лекарства, радиоактивные материалы, объекты флоры и фауны, а также высокочастотные радио-электронные устройства и средства связи - в частности, системы радиолокации, кабельного телевидения и средства мобильной связи частотой выше 9 килогерц. Кроме того, в декларации будет необходимо приводить более подробные сведения о ввозимом или вывозимом транспортном средстве - вид машинs, год выпуска, объем двигателя, номера двигателя, кузова и шасси. В России новая форма декларации наряду с прежней применялась с 1 августа лишь на московских таможенных пунктах: Шереметьево, во Внуково, в Домодедово, Чкаловском и на железнодорожных вокзалах. Теперь она вводится на всей территории страны. Преимуществом новой декларации, пояснили в ГТК, является то, что она будет единой на территории 11 государств СНГ, подписавших соглашение, к которому пока не присоединилась лишь Армения, но с ней ""сейчас ведутся конструктивные переговоры"". Новая таможенная декларация уже вступила в силу на территории Украины и Белоруссии, другие государства СНГ заявили о своей готовности ввести ее на своей территории до концатекущего года.",                
    "Указом президента России Бориса Ельцина внесены изменения в  существующую структуру Федеральной службы безопасности РФ, утвержденную в июле прошлого года. Как говорится в поступившем сегодня сообщении Центра общественных связей ФСБ, в соответствии с Основами (Концепцией) государственной политики Российской Федерации по военномустроительству на период до 2005 года, на базе Департамента по борьбе с терроризмом и Управления конституционной безопасности ФСБ создан Департамент по защите конституционного строя и борьбе с терроризмом. В составе департамента организуются три управления с четко определенной компетенцией. В ФСБ отмечают, что ""в современных условиях для российскойгосударственности имеют приоритетное значение вопросы защитыконституционного строя, сохранения целостности страны, борьбыс терроризмом и всеми формами экстремизма, а также разведывательно-подрывной деятельностью спецслужб и организаций иностранных государств"". Как подчеркивается в сообщении, ""органам безопасности в решении данных проблем отведена особая роль"".",          
    "Сегодня областной центр Сахалина и Курил получил статус очага распространения холеры. Как сообщает ИТАР-ТАСС со ссылкой на пресс-центр администрации Сахалинской области, в лечебных учреждениях Южно-Сахалинска уже находятсятся 5 горожан, причем у двоих из них болезнь проходит в средне-тяжелой форме. Специалисты Госсанэпиднадзора по Сахалинской области и Хабаровской противочумной станции считают, что возможным источником заражения стала река Хомутовка в военном городке и слабосоленная красная рыба собственного приготовления, которую жители промывали в этой реке. Решением губернатора Сахалинской области Игоря Фархутдинова сегодня в Южно-Сахалинске начато развертывание специальных госпиталей и изоляторов для локализации и ликвидации очага холеры. Кроме того, на заседании санитарно-противоэпидемической комиссии было принято решение о трехкратном бактериологическом обследовании всех больных, обратившихся в медицинские учреждения города с жалобами на кишечные расстройства. В Южно-Сахалинске установлен запрет на купание и водопользование в неустановленных местах.",       
    "Московский пиво-безалкогольный комбинат ""Очаково"" представит традиционный для россиян, но принципиально новый для рынка напиток - рассол. Как пишет газета ""Ведомости"", он будет выпускаться в алюминиевых банках и пол-литровых бутылках. Будет два вида рассола - свекольный и капустный. В продаже рассол появится к майским праздникам, когда обычно резко возрастает потребление алкогольных напитков. Впрочем, удовольствие выпить с утра рассолу будет не из дешевых, как пишет газета, новый напиток будет дороже многих сортов пива и будет стоить около 12 рублей. Рассол планируется слегка газировать, чтобы сделать его вкус ядренее. По мнению специалистов, рассол очень полезный напиток - в нем содержатся молочная кислота и соль, которые способствуют борьбе с похмельем.",             
    "16 декабря российское правительство одобрило проект концепции государственной поддержки экономического и социального развития районов Севера, сообщает агентство РИА ""Новости"". В настоящее время правительство рассматривает различные варианты разрешения ситуации, сложившейся в российских северных регионах. Как заявил на заседании правительства премьер-министр Владимир Путин, ""необходимо рационально использовать ресурсы Севера"". По мнению премьера, эти территории представляют собой ""огромный трудовой и ресурсный потенциал страны"". В частности, на территориях Крайнего Севера добывается около 75 процентов нефти, 90 процентов газа и 100 процентов алмазов. Однако без государственной поддержки экономическое и социальное развитие районов Крайнего Севера невозможно, убежден российский премьер. Следовательно, необходимо выработать комплексную систему эффективных мер по развитию экономики и социальной сферы районов Крайнего Севера. На переход к ""сбалансированному социально-экономическому развитию"" Севера в одобренной правительством концепции отведено 15 лет.",                  
    "16 декабря в Хельсинки завершился российско-финский семинар ""ЕС-Россия: усиление борьбы с международной организованной преступностью"". В ходе переговоров, посвященных развитию сотрудничества правоохранительных органов двух стран в приграничных регионах, стороны утвердили и подписали совместный план борьбы с организованной преступностью ""Граница-2000"". Кроме того, в рамках встречи было подписано двустороннее соглашение между министерствами внутренних дел России и Финляндии о сотрудничестве. Как заявил журналистам РИА ""Новости"" глава российской делегации, министр внутренних дел РФ Владимир Рушайло, обе стороны стремились найти оптимальные пути решения проблем борьбы с преступностью и международным терроризмом. По словам Рушайло, Москва придает большое значение достигнутым договоренностям. В ближайшее время эти договоренности начнут осуществляться, сообщает агентство Reuters. Участники семинара не поднимали вопрос об обстановке на Северном Кавказе. По утверждению Владимира Рушайло, интерес к этой теме проявляли лишь журналисты."
]

# Анализ
print("Анализ эмоциональной окраски текстов:")
results = analyze_emotions(texts)
for i, result in enumerate(results):
    print(f"\nТекст {i+1}:")
    print(f"Текст: {result['text']}")
    print(f"Предсказанная эмоция: {result['predicted_emotion']}")
    print(f"Вероятности: {result['probabilities']}")

Анализ эмоциональной окраски текстов:


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Модель загружена успешно

Текст 1:
Текст: Бои у Сопоцкина и Друскеник закончились отступлением германцев. Неприятель, приблизившись с севера к...
Предсказанная эмоция: позитивный
Вероятности: [0.25387555 0.23790158 0.25597826 0.25224465]

Текст 2:
Текст: Министерство народного просвещения, в виду происходящих чрезвычайных событий, признало соответственн...
Предсказанная эмоция: смешанный
Вероятности: [0.25189167 0.2502444  0.22914967 0.26871428]

Текст 3:
Текст: Штабс-капитан П. Н. Нестеров на днях, увидев в районе Желтиева, в Галиции, летящий над нашим располо...
Предсказанная эмоция: нейтральный
Вероятности: [0.26264125 0.2582057  0.23693375 0.2422193 ]

Текст 4:
Текст: Фотограф-корреспондент Daily Mirror рассказывает случай, который порадует всех друзей животных. Лейт...
Предсказанная эмоция: позитивный
Вероятности: [0.24621901 0.251701   0.2555645  0.24651542]

Текст 5:
Текст: Лица, приехавшие в Варшаву из Люблина, передают, что туда доставлен «швабский зверь» — взятый в плен...
Пр

In [7]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from chonkie import SentenceChunker

def load_model_for_inference():
    """Загружает модель и токенизатор для анализа эмоций."""
    model_name = 'cointegrated/rubert-tiny2'
    model = BertForSequenceClassification.from_pretrained(model_name, num_labels=4)
    tokenizer = BertTokenizer.from_pretrained(model_name)
    return model, tokenizer

def analyze_text_chunks_detailed(text, model, tokenizer, chunker):
    """Анализирует тональность текста с разбивкой на чанки, возвращая детальные результаты."""
    # 1. Разбиваем текст на чанки
    chunks = chunker(text)
    
    # 2. Анализируем каждый чанк
    chunk_results = []
    for chunk in chunks:
        chunk_text = chunk.text
        if not chunk_text.strip():
            continue
            
        inputs = tokenizer(chunk_text, return_tensors="pt", truncation=True, padding=True, max_length=512)
        
        with torch.no_grad():
            outputs = model(**inputs)
        
        probabilities = torch.softmax(outputs.logits, dim=1)
        chunk_probs = probabilities.numpy()[0]
        
        # Определяем эмоцию для этого чанка
        emotion_idx = chunk_probs.argmax()
        emotions = ['нейтральный', 'негативный', 'позитивный', 'смешанный']
        
        chunk_results.append({
            'text': chunk_text,
            'probs': chunk_probs,
            'emotion': emotions[emotion_idx],
            'confidence': float(chunk_probs[emotion_idx])  # Уверенность в этой эмоции
        })
    
    return chunk_results

def get_final_emotion(chunk_results):
    """Определяет итоговую эмоцию на основе анализа всех чанков."""
    if not chunk_results:
        return 'нейтральный', {}, []
    
    # 1. Собираем статистику по эмоциям в чанках
    emotion_counts = {'нейтральный': 0, 'негативный': 0, 'позитивный': 0, 'смешанный': 0}
    emotion_confidences = {'нейтральный': [], 'негативный': [], 'позитивный': [], 'смешанный': []}
    
    for chunk in chunk_results:
        emotion = chunk['emotion']
        emotion_counts[emotion] += 1
        emotion_confidences[emotion].append(chunk['confidence'])
    
    # 2. Находим наиболее частую эмоцию
    max_count = max(emotion_counts.values())
    most_common_emotions = [emotion for emotion, count in emotion_counts.items() if count == max_count]
    
    # 3. Если одна эмоция встречается чаще всех
    if len(most_common_emotions) == 1:
        final_emotion = most_common_emotions[0]
    else:
        # 4. При равенстве выбираем с наибольшей средней уверенностью
        avg_confidences = {}
        for emotion in most_common_emotions:
            if emotion_confidences[emotion]:
                avg_confidences[emotion] = sum(emotion_confidences[emotion]) / len(emotion_confidences[emotion])
            else:
                avg_confidences[emotion] = 0
        
        final_emotion = max(avg_confidences.items(), key=lambda x: x[1])[0]
    
    # 5. Собираем средние вероятности для каждой эмоции
    avg_probs = [0, 0, 0, 0]
    if chunk_results:
        all_probs = [chunk['probs'] for chunk in chunk_results]
        avg_probs = sum(all_probs) / len(all_probs)
    
    emotion_probs = {
        'нейтральный': float(avg_probs[0]),
        'негативный': float(avg_probs[1]),
        'позитивный': float(avg_probs[2]),
        'смешанный': float(avg_probs[3])
    }
    
    return final_emotion, emotion_probs, chunk_results

def analyze_news_sentiment_detailed(texts):
    """Анализирует тональность новостей с детальным выводом по чанкам."""
    # Загружаем модель
    model, tokenizer = load_model_for_inference()
    model.eval()
    
    # Инициализируем чанкер
    chunker = SentenceChunker()  # Разбиваем по предложениям
    
    # Анализируем каждый текст
    results = []
    for text in texts:
        # Анализируем чанки
        chunk_results = analyze_text_chunks_detailed(text, model, tokenizer, chunker)
        
        # Определяем итоговую эмоцию
        final_emotion, emotion_probs, detailed_chunks = get_final_emotion(chunk_results)
        
        # Эмодзи для наглядности
        emotion_icons = {
            'нейтральный': '😐',
            'негативный': '😠',
            'позитивный': '😊',
            'смешанный': '🤔'
        }
        
        results.append({
            'text_preview': text[:80] + '...' if len(text) > 80 else text,
            'final_emotion': final_emotion,
            'emotion_icon': emotion_icons.get(final_emotion, '❓'),
            'emotion_probs': emotion_probs,
            'chunks_analyzed': len(chunk_results),
            'chunk_details': detailed_chunks[:3]  # Показываем первые 3 чанка для краткости
        })
    
    return results

# Ваши тексты
texts = [
    "Бои у Сопоцкина и Друскеник закончились отступлением германцев. Неприятель, приблизившись с севера к Осовцу начал артиллерийскую борьбу с крепостью. В артиллерийском бою принимают участие тяжелые калибры. С раннего утра 14 сентября огонь достиг значительного напряжения. Попытка германской пехоты пробиться ближе к крепости отражена. В Галиции мы заняли Дембицу. Большая колонна, отступавшая по шоссе от Перемышля к Саноку, обстреливалась с высот нашей батареей и бежала, бросив парки, обоз и автомобили. Вылазки гарнизона Перемышля остаются безуспешными. При продолжающемся отступлении австрийцев обнаруживается полное перемешивание их частей, захватываются новые партии пленных, орудия и прочая материальная часть. На перевале Ужок мы разбили неприятельский отряд, взяли его артиллерию и много пленных и, продолжая преследовать, вступили в пределы Венгрии.",           
    "Министерство народного просвещения, в виду происходящих чрезвычайных событий, признало соответственным в день годовщины со дня рождения М.Ю. Лермонтова (2-го октября 1914 года) ограничиться совершением в учебных заведениях панихиды по поэту, отложив празднование юбилея до более благоприятного времени.",           
    "Штабс-капитан П. Н. Нестеров на днях, увидев в районе Желтиева, в Галиции, летящий над нашим расположением австрийский аэроплан, собиравшийся бросить бомбы, взлетел на воздух, атаковал неприятеля и протаранил неприятельский аппарат, предотвратив жертвы в наших войсках. Сам Нестеров при этом погиб смертью героя. По словам доставленных в Киев пленных австрийских офицеров, всей неприятельской армии хорошо известно имя Нестерова. Во время воздушных разведок русских авиаторов австрийцы всегда безошибочно определяли, каким аппаратом управлял Нестеров. Когда показывался аэроплан-птица, красиво и вольно паривший в воздухе, австрийцы указывали — Das ist Nesteroff! Австрийцы боялись покойного, и все их усилия были направлены к прекращению его деятельности. За задержание отважного летчика была объявлена большая премия. Нестеров погиб в 27 лет. После Нестерова остались жена и двое детей — девочка, 5-ти лет, и мальчик, 3-х лет. Иллюстрированный журнал «Искры» № 35, сентябрь 1914 года  Песнь о Нестерове В безбрежности неба, в бескрайности ясной пустыниСражались лишь птицы и гибли лишь птицы доныне.Так птице подобный, полет свой направив могучий,Унесся пилот легкокрылый за темные тучи.А там, на земле, колыхаясь, знамена шумели, И лязгали сабли, и пушки зловеще гремели.И пламенно богу войны посылал он молитвы,И очи горели и звали и жаждали битвы…Величие духа в равнине обманчиво-зыбкой.Презрение смерти под светлою солнца улыбкой…Мольбы не напрасны. Велению рока послушный,Противник уж реет кругами в стихии воздушнойИ дрогнуло сердце. Рука, как пружина стальная,На руль налегла. Встрепенулась машина живая…Чти, родина, память героя! Решенье созрелоМгновенье – и врезался дерзко он в хрупкое тело.Пощады не знает стихия. Победа наградой…И пали противники грузной и страшной громадой…В безбрежности неба, в бескрайности ясной пустыниСражались лишь птицы и гибли лишь птицы доныне. Журнал «Нива» №37, сентябрь 1914 года",                  
    "Фотограф-корреспондент Daily Mirror рассказывает случай, который порадует всех друзей животных. Лейтенант бельгийской артиллерии, руководивший обороной одного из фортов Льежа, ни за что не хотел расстаться с своей собакой — бульдогом. Когда пруссаки пробрались между фортов в самый город, офицеру пришло в голову доверить бульдогу письмо, в котором он посылал успокоительную весть своим родителям. Благородный пес честно исполнил свою миссию. Десять часов спустя бульдог проник обратно в форт и принес ответ. С этого момента бульдог стал настоящим гонцом. Много раз пробирался он через линии германских войск, неся на себе спрятанными в ошейнике шифрованные депеши.",           
    "Лица, приехавшие в Варшаву из Люблина, передают, что туда доставлен «швабский зверь» — взятый в плен прусский майор Прейскер, бывший комендант Калиша. Это — здоровый детина, с типично прусским наглым лицом и мутными глазами. Когда этого «зверя» вели с партией пленных по улице и из толпы раздались крики и проклятия, Прейскер трусливо замешался в толпу пленных, стараясь скрыться. Зато в заключении, под конвоем наших солдат, Прейскер принял пришедшего к нему русского офицера сидя, нагло развалившись в кресле. Конвойные «помогли» ему подняться и вытянуться во фронт. Кроме Прейскера, нашими войсками взят в плен и другой «швабский зверь» — капитан Шмидт, живьем сжигавший жителей деревень, запирая их в хатах. Журнал «Нива» №38 от 20 сентября 1914 года",                   
    "Как стало известно агентству Ассошиэйтед Пресс, экипаж последней экспедиции на станцию ""Мир"" считает ее способной выйти из-под контроля.Командир Виктор Афанасьев сказал: ""Мы чувствуем себя хорошо, но грустим, что оставили станцию летать в беспилотном режиме."" Gazeta Ru подробно писала о том, что финансирование программы ""Мир"" закрыто, но конструкторы не спешат топить ее в Тихом океане, надеясь на изыскание внебюджетных средств.Они надеются, что правительство оплатит финальную экспедицию на ""Мир"" в феврале или марте 2000 года. Новая экспедиция должна постепенно перевести станцию на низкую орбиту, после чего 140- тонный ""Мир"" в основном сгорит в атмосфере, а незначительные его части упадут в Тихий океан. Но нет никакой гарантии, что пережившая столько отказов оборудования конструкция поведет себя адекватно командам. Афанасьев перечислил потенциально опасные элементы ""Мира"" на пресс- конференции, состоявшейся в понедельник в Звездном Городке. ""Может упасть давление в кабине,- сказал он,- и это станет серьезным препятствием для работы следующей экспедиции. Слишком велика роль ""человеческого фактора"" , элементарная ошибка может привести как к потере топлива, так и к потере ориентации."" Американское космическое агентство по аэронавтике и космическому пространству предлагало России затопить ""Мир"", а его ресурсы перевести на международную космическую станцию. Но Россия не хочет расставаться с легендарным символом.",     
    "В зале игровых автоматов в третьем ярусе подземного комплекса ""Охотный ряд"" на Манежной площади в центре Москвы около 20.00 прогремел мощный взрыв. По данным ФСБ, погибли не менее четырех человек, 20 человек ранены. Однако уточненная оценка числа пострадавших в результате этого взрыва может достигнуть ста человек. Агентство ИТАР-ТАСС, в сообщении от 21.15 со ссылкой на источники в ГУВД Москвы, говорит только о 30 раненых, в том числе о двух пострадавших в тяжелом состоянии. Однако число пострадавших в результате этого взрыва может составить до ста человек. По данным РИА ""Новости"", Боткинская больница, Институт им. Склифосовского, 1-ая Градская, 36-ая и 64-ая горбольница работают только на прием пострадавших. С места происшествия запросили 20 машин скорой помощи. Из торгового центра эвакуированы все посетитетели. Среди причин происшедшего называют террористический акт, связанный с последними событиями в Дагестане, однако, по сообщению ОРТ, не исключается и версия об аварии (взорвался игровой автомат). Более точных данных о причнах взрыва и количестве жертв представители УФСБ называть не торопятся.",                  
    "Япония приняла решение разморозить кредиты России на сумму в полтора миллиарда долларов. Об этом заявил сегодня после встречи с премьер-министром Японии вице-премьер правительства России Виктор Христенко. Он прибыл в Токио во главе делегации для участия в заседании  российско-японской межправительственной комиссии по торгово-экономическим  вопросам, которое  начнется в  среду  в Токио. Один из участников подготовки этого заседания сообщил ""Интерфаксу"", что Христенко постарается на конкретных макроэкономических показателях  за последние  месяцы доказать, что экономика России становится более стабильной и убедить Японию в своей платежеспособности и партнерской надежности. ""Если на заседании  комиссии японская сторона будет удовлетворена этими данными, то многие российско-японские проекты по улучшению  экономики дальневосточного побережья, примут более реальные очертания"", - подчеркнул источник. Сегодняшнее сообщение о разморозке кредита показывает, что приведенные Христенко выкладки оказались убедительными для японцев. В  отношении   Японии  еще   со  времен СССР  осталаськоммерческая задолженность  японским фирмам,  общий объем которой составляет около $1,1 миллиардов. Основная задолженность перед крупными фирмами, объединившимися  в Токийский клуб, составляет около $925 миллионов. Общий же объем внешнего долга России в настоящее время  составляет около $150 миллиардов, и платежи по его обслуживанию в соответствии  с графиком  должны были  бы составить  около  90% доходов федерального бюджета.",               
    "Британцы отмечают сегодня скорбную дату - вторуюгодовщину со дня смерти принцессы Дианы. Она погибла вавтомобильной катастрофе в парижском туннеле в ранние утренниечасы 31 августа 1997 года. С утра к золоченым воротамКенсингтонского дворца, где жила Диана, приходят ее поклонники.В чугунные завитки ограды вплетаются сотни букетов. Людиоставляют дары принцессе - коробки конфет, плюшевых мишек,открытки со посвященными ей стихами, стенгазеты. Вчера и сегодня болееТысячи англичан подписали петицию с требованием к властямускорить сооружение мемориала принцессе. Пока Мемориальныйкомитет собирается переоборудовать в ее честь детскую игровуюплощадку, которая находится на территории Кенсингтонского паркаи создать посвященную ей ""прогулочную дорожку"" вдоль похоронного маршрута. Мемориальный фонд имени принцессы Дианы собрал в видепожертвований населения более 85 миллонов фунтов (140 миллионов долларов).Средичленов правления фонда нет согласия относительно использованиясредств. Они заняты не столько помощью больным иобделенным, как завещала принцесса, сколько в основном судебнойсхваткой за авторские права на ее, Дианы, изображения. Королевская семья отметит траурную годовщину сдержаннойпоминальной церемонией в шотландском замке Балморал, где будутприсутствовать сыновья Дианы - 17-летний Вильям и 14-летнийГарри, их отец принц Чарльз. Родственники Дианы соберутсяотдельно в Олторпе, поместье ее брата графа Чарльза Спенсера. Впоместье создан мемориальный музей Дианы, который в этом годупосетило 142 тысячи человек. Ожидается, что в ближайшие дни будут оглашеныокончательные результаты расследования аварии в Париже.Британские газеты уже получили этот документ. Никаких сенсацийтам нет. Единственная причина катастрофы - опьянение водителя ипревышение скорости, пришли к заключению французскиеследователи. Содержание алкоголя в крови водителя лимузина Генри Пола в три раза превосходило норму- 1,8 грамма на литр. Фиат ""Уно"", якобы послуживший причиной трагедии, так и не найден- в Париже таких автомобилей более сорока тысяч. Не будет предъявлено никаких обвиненийфотографам-папарацци, преследовавшим ""Мерседес"" Дианы на мотоциклах. Письма Дианы к бывшему любовнику,майору Джеймсу Хьюитту, могут быть вскоре опубликованы отдельной книгой. Хьюитт нарушил клятву никогда не использовать для печати 65 писем принцессы, написанных ею после того, как ее брак с принцем Чарльзом развалился. В письмах королевская семья характеризуется весьма нелицеприятно.",              
    "С 1 сентября на всей территории России вводится новая форма таможенной декларации для физических лиц. Как пояснили в Государственном таможенном комитете (ГТК), форма декларации, которая была одобрена Советом руководителей таможенных служб государств-участников Содружества, является единой на территории всех стран СНГ, за исключением Армении. По данным ИТАР-ТАСС, в новой декларации содержится более подробный перечень товаров, подлежащих обязательному декларированию. Так, впервые в их число включены ядовитые и сильно действующие вещества и лекарства, радиоактивные материалы, объекты флоры и фауны, а также высокочастотные радио-электронные устройства и средства связи - в частности, системы радиолокации, кабельного телевидения и средства мобильной связи частотой выше 9 килогерц. Кроме того, в декларации будет необходимо приводить более подробные сведения о ввозимом или вывозимом транспортном средстве - вид машинs, год выпуска, объем двигателя, номера двигателя, кузова и шасси. В России новая форма декларации наряду с прежней применялась с 1 августа лишь на московских таможенных пунктах: Шереметьево, во Внуково, в Домодедово, Чкаловском и на железнодорожных вокзалах. Теперь она вводится на всей территории страны. Преимуществом новой декларации, пояснили в ГТК, является то, что она будет единой на территории 11 государств СНГ, подписавших соглашение, к которому пока не присоединилась лишь Армения, но с ней ""сейчас ведутся конструктивные переговоры"". Новая таможенная декларация уже вступила в силу на территории Украины и Белоруссии, другие государства СНГ заявили о своей готовности ввести ее на своей территории до концатекущего года.",                
    "Указом президента России Бориса Ельцина внесены изменения в  существующую структуру Федеральной службы безопасности РФ, утвержденную в июле прошлого года. Как говорится в поступившем сегодня сообщении Центра общественных связей ФСБ, в соответствии с Основами (Концепцией) государственной политики Российской Федерации по военномустроительству на период до 2005 года, на базе Департамента по борьбе с терроризмом и Управления конституционной безопасности ФСБ создан Департамент по защите конституционного строя и борьбе с терроризмом. В составе департамента организуются три управления с четко определенной компетенцией. В ФСБ отмечают, что ""в современных условиях для российскойгосударственности имеют приоритетное значение вопросы защитыконституционного строя, сохранения целостности страны, борьбыс терроризмом и всеми формами экстремизма, а также разведывательно-подрывной деятельностью спецслужб и организаций иностранных государств"". Как подчеркивается в сообщении, ""органам безопасности в решении данных проблем отведена особая роль"".",          
    "Сегодня областной центр Сахалина и Курил получил статус очага распространения холеры. Как сообщает ИТАР-ТАСС со ссылкой на пресс-центр администрации Сахалинской области, в лечебных учреждениях Южно-Сахалинска уже находятсятся 5 горожан, причем у двоих из них болезнь проходит в средне-тяжелой форме. Специалисты Госсанэпиднадзора по Сахалинской области и Хабаровской противочумной станции считают, что возможным источником заражения стала река Хомутовка в военном городке и слабосоленная красная рыба собственного приготовления, которую жители промывали в этой реке. Решением губернатора Сахалинской области Игоря Фархутдинова сегодня в Южно-Сахалинске начато развертывание специальных госпиталей и изоляторов для локализации и ликвидации очага холеры. Кроме того, на заседании санитарно-противоэпидемической комиссии было принято решение о трехкратном бактериологическом обследовании всех больных, обратившихся в медицинские учреждения города с жалобами на кишечные расстройства. В Южно-Сахалинске установлен запрет на купание и водопользование в неустановленных местах.",       
    "Московский пиво-безалкогольный комбинат ""Очаково"" представит традиционный для россиян, но принципиально новый для рынка напиток - рассол. Как пишет газета ""Ведомости"", он будет выпускаться в алюминиевых банках и пол-литровых бутылках. Будет два вида рассола - свекольный и капустный. В продаже рассол появится к майским праздникам, когда обычно резко возрастает потребление алкогольных напитков. Впрочем, удовольствие выпить с утра рассолу будет не из дешевых, как пишет газета, новый напиток будет дороже многих сортов пива и будет стоить около 12 рублей. Рассол планируется слегка газировать, чтобы сделать его вкус ядренее. По мнению специалистов, рассол очень полезный напиток - в нем содержатся молочная кислота и соль, которые способствуют борьбе с похмельем.",             
    "16 декабря российское правительство одобрило проект концепции государственной поддержки экономического и социального развития районов Севера, сообщает агентство РИА ""Новости"". В настоящее время правительство рассматривает различные варианты разрешения ситуации, сложившейся в российских северных регионах. Как заявил на заседании правительства премьер-министр Владимир Путин, ""необходимо рационально использовать ресурсы Севера"". По мнению премьера, эти территории представляют собой ""огромный трудовой и ресурсный потенциал страны"". В частности, на территориях Крайнего Севера добывается около 75 процентов нефти, 90 процентов газа и 100 процентов алмазов. Однако без государственной поддержки экономическое и социальное развитие районов Крайнего Севера невозможно, убежден российский премьер. Следовательно, необходимо выработать комплексную систему эффективных мер по развитию экономики и социальной сферы районов Крайнего Севера. На переход к ""сбалансированному социально-экономическому развитию"" Севера в одобренной правительством концепции отведено 15 лет.",                  
    "16 декабря в Хельсинки завершился российско-финский семинар ""ЕС-Россия: усиление борьбы с международной организованной преступностью"". В ходе переговоров, посвященных развитию сотрудничества правоохранительных органов двух стран в приграничных регионах, стороны утвердили и подписали совместный план борьбы с организованной преступностью ""Граница-2000"". Кроме того, в рамках встречи было подписано двустороннее соглашение между министерствами внутренних дел России и Финляндии о сотрудничестве. Как заявил журналистам РИА ""Новости"" глава российской делегации, министр внутренних дел РФ Владимир Рушайло, обе стороны стремились найти оптимальные пути решения проблем борьбы с преступностью и международным терроризмом. По словам Рушайло, Москва придает большое значение достигнутым договоренностям. В ближайшее время эти договоренности начнут осуществляться, сообщает агентство Reuters. Участники семинара не поднимали вопрос об обстановке на Северном Кавказе. По утверждению Владимира Рушайло, интерес к этой теме проявляли лишь журналисты."
]

# Запускаем анализ
print("=" * 70)
print("ДЕТАЛЬНЫЙ АНАЛИЗ ТОНАЛЬНОСТИ С РАЗБИВКОЙ НА ЧАНКИ")
print("=" * 70)

results = analyze_news_sentiment_detailed(texts)

for i, result in enumerate(results):
    print(f"\n{'═'*70}")
    print(f"НОВОСТЬ #{i+1}: {result['emotion_icon']} {result['final_emotion'].upper()}")
    print(f"Текст: {result['text_preview']}")
    print(f"Проанализировано чанков: {result['chunks_analyzed']}")
    
    # Вероятности по всем эмоциям
    print("\nВероятности эмоций по всему тексту:")
    for emotion, prob in result['emotion_probs'].items():
        print(f"  {emotion}: {prob:.1%}")
    
    # Детали по первым чанкам
    if result['chunk_details']:
        print(f"\nАнализ первых {len(result['chunk_details'])} чанков:")
        for j, chunk in enumerate(result['chunk_details'], 1):
            print(f"  Чанк {j}: \"{chunk['text'][:60]}...\"")
            print(f"    → {chunk['emotion']} (уверенность: {chunk['confidence']:.1%})")
    
    print(f"{'═'*70}")

# Сводная статистика
print(f"\n{'='*70}")
print("СВОДНАЯ СТАТИСТИКА:")
emotion_counts = {}
for result in results:
    emotion = result['final_emotion']
    emotion_counts[emotion] = emotion_counts.get(emotion, 0) + 1

for emotion, count in emotion_counts.items():
    icon = {'нейтральный': '😐', 'негативный': '😠', 'позитивный': '😊', 'смешанный': '🤔'}.get(emotion, '❓')
    print(f"  {icon} {emotion}: {count} новостей")
print(f"{'='*70}")

ДЕТАЛЬНЫЙ АНАЛИЗ ТОНАЛЬНОСТИ С РАЗБИВКОЙ НА ЧАНКИ


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



══════════════════════════════════════════════════════════════════════
НОВОСТЬ #1: 😐 НЕЙТРАЛЬНЫЙ
Текст: Бои у Сопоцкина и Друскеник закончились отступлением германцев. Неприятель, приб...
Проанализировано чанков: 1

Вероятности эмоций по всему тексту:
  нейтральный: 25.4%
  негативный: 25.1%
  позитивный: 24.4%
  смешанный: 25.1%

Анализ первых 1 чанков:
  Чанк 1: "Бои у Сопоцкина и Друскеник закончились отступлением германц..."
    → нейтральный (уверенность: 25.4%)
══════════════════════════════════════════════════════════════════════

══════════════════════════════════════════════════════════════════════
НОВОСТЬ #2: 😠 НЕГАТИВНЫЙ
Текст: Министерство народного просвещения, в виду происходящих чрезвычайных событий, пр...
Проанализировано чанков: 1

Вероятности эмоций по всему тексту:
  нейтральный: 23.7%
  негативный: 26.4%
  позитивный: 25.4%
  смешанный: 24.4%

Анализ первых 1 чанков:
  Чанк 1: "Министерство народного просвещения, в виду происходящих чрез..."
    → негативный (увере